# Universidad del Desarrollo
## Magister Data Science - Macroeconomía
### Grupo N°2: (Región de los Lagos)
* Jose Pedro
* Cristian
* Paul
* Max
* Hernan

## Calculo del Micro Indice de productos

**Objetivo**

El objetivo de este notebook es el calculo de la serie de precios de producto segun el sigueinte algoritmo.



$ 
\begin{equation*} 
p_i^t = \prod_{j=1}^n (\frac{v_j^t}{v_j^{t-1}})^{1/n} 
\end{equation*} 
$

Donde:
* $ p_i^t $ = Precio del producto i en el momento t
* $ p_i^0 $ = Precio del producto 0 en el mes base **(2014/08=100)**
* $ w_i   $ = Ponderación del producto i en la canasta
* $ v_j^t $ = Precio de la variaedad j en el momento t
* N = Numero de productos
* n = Numero de variedades

Dados los siguientes supuestos:
* n = 2
* t = 2014/09

Entonces:

P = ([ p_var1(2014-09) / p_var1(2014-08) ]$**$(1/2)) $*$ ([ p_var2(2014-09) / p_var2(2014-08) ]$**$(1/2))

In [29]:
import warnings
warnings.simplefilter(action='ignore')

In [30]:
import pandas as pd
import numpy as np
import calendar
import datetime

%matplotlib inline

### 1. Se cargan el dataset de productos

In [31]:
df = pd.read_csv('output/canasta_ipc.csv')

In [32]:
df.head()

,Momento,Sector,Producto,Variedad,Promedio
0,2014-08,Frutillar,Aceite,Maravilla,1807.484158
1,2014-09,Frutillar,Aceite,Maravilla,1680.222200
2,2014-10,Frutillar,Aceite,Maravilla,1651.904700
3,2014-11,Frutillar,Aceite,Maravilla,1807.484158
4,2014-12,Frutillar,Aceite,Maravilla,1807.484158


In [33]:
df.Producto.value_counts()

Carnes       12692
Hortaliza     9587
Frutas        7951
Lacteos       2350
Pan           1925
Legumbre      1410
Cereales       940
Huevos         470
Aceite         470
Harina         470
Azucar         235
Name: Producto, dtype: int64

### 2.- Transformamos Monento a un tipo de dato fecha para iterar sobre el periodo

In [34]:
df["Momento"] = pd.to_datetime(df["Momento"])

In [35]:
df.head()

,Momento,Sector,Producto,Variedad,Promedio
0,2014-08-01,Frutillar,Aceite,Maravilla,1807.484158
1,2014-09-01,Frutillar,Aceite,Maravilla,1680.222200
2,2014-10-01,Frutillar,Aceite,Maravilla,1651.904700
3,2014-11-01,Frutillar,Aceite,Maravilla,1807.484158
4,2014-12-01,Frutillar,Aceite,Maravilla,1807.484158


* Años base:

In [36]:
base = datetime.date(2014, 8, 1)

### 3. Creamos una funcion para restar un mes al periodo de estudio

In [37]:
def add_months(sourcedate, months):
    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year, month)[1])
    return datetime.date(year, month, day)

### 4. Creamos función para calculo de la serie factores de producto

In [38]:
def GetSerieProducto(df_input, group):
    v = []
    for name, g in group: 
        
        factor = 1
        n = g.Promedio.count()

        if g.Momento.iloc[0].date() > base:
            for row_num, row in g.iterrows(): 

                Pro1 = row["Promedio"]
                Mom2 = add_months(row["Momento"], -1)

                Pro2 = df_input[(df_input.Sector==row["Sector"]) & (df_input.Momento == Mom2) & (df_input.Variedad == row["Variedad"])]["Promedio"].iloc[0]

                factor *= (Pro1/Pro2)**(1/n)

        name = name + (factor,)  
        v.append(name)
    
    return pd.DataFrame(v, columns=['Momento', 'Sector', 'Producto', 'Factor'])

### 5. Calculo de factores de productos

#### 5.1 Aceite

In [39]:
df_aceite = df[df.Producto=="Aceite"]
group = df_aceite.groupby(['Momento', 'Sector', 'Producto'])

In [40]:
dfs_aceite = GetSerieProducto(df_aceite, group)

In [41]:
dfs_aceite.sample(3)

,Momento,Sector,Producto,Factor
92,2016-02-01,Osorno,Aceite,0.992594
225,2018-05-01,Frutillar,Aceite,0.941019
81,2015-12-01,Llanquihue,Aceite,1.026753


#### 5.2 Harina

In [42]:
df_harina = df[df.Producto=="Harina"]
group = df_harina.groupby(['Momento', 'Sector', 'Producto'])

In [43]:
dfs_harina = GetSerieProducto(df_harina, group)

In [44]:
dfs_harina.sample(3)

,Momento,Sector,Producto,Factor
157,2017-03-01,Osorno,Harina,0.981360
27,2015-01-01,Osorno,Harina,1.002099
0,2014-08-01,Frutillar,Harina,1.000000


#### 5.3. Cereales

In [45]:
df_cereales = df[df.Producto=="Cereales"]
group = df_cereales.groupby(['Momento', 'Sector', 'Producto'])

In [46]:
dfs_cereales = GetSerieProducto(df_cereales, group)

In [47]:
dfs_cereales.sample(3)

,Momento,Sector,Producto,Factor
92,2016-02-01,Osorno,Cereales,1.014363
93,2016-02-01,Puerto Montt,Cereales,1.000157
71,2015-10-01,Llanquihue,Cereales,0.996155


#### 5.4 Azucar

In [48]:
df_azucar = df[df.Producto=="Azucar"]

In [49]:
group = df_azucar.groupby(['Momento', 'Sector', 'Producto'])

In [50]:
dfs_azucar = GetSerieProducto(df_azucar, group)

In [51]:
dfs_azucar.sample(3)

,Momento,Sector,Producto,Factor
227,2018-05-01,Osorno,Azucar,0.975521
144,2016-12-01,Puerto Varas,Azucar,0.974602
185,2017-09-01,Frutillar,Azucar,0.996897


#### 5.5 Carne

In [52]:
df_carnes = df[df.Producto=="Carnes"]

In [53]:
group = df_carnes.groupby(['Momento', 'Sector', 'Producto'])

In [54]:
dfs_carnes = GetSerieProducto(df_carnes, group)

In [55]:
dfs_carnes.sample(3)

,Momento,Sector,Producto,Factor
147,2017-01-01,Osorno,Carnes,1.029974
154,2017-02-01,Puerto Varas,Carnes,1.036814
207,2018-01-01,Osorno,Carnes,0.953310


#### 5.6 Hortaliza

In [56]:
df_hortaliza = df[df.Producto=="Hortaliza"]

In [57]:
group = df_hortaliza.groupby(['Momento', 'Sector', 'Producto'])

In [58]:
dfs_hortaliza = GetSerieProducto(df_hortaliza, group)

In [59]:
dfs_hortaliza.sample(3)

,Momento,Sector,Producto,Factor
63,2015-08-01,Puerto Montt,Hortaliza,1.024020
4,2014-08-01,Puerto Varas,Hortaliza,1.000000
62,2015-08-01,Osorno,Hortaliza,0.981551


#### 5.7 Frutas

In [60]:
df_frutas = df[df.Producto=="Frutas"]

In [61]:
group = df_frutas.groupby(['Momento', 'Sector', 'Producto'])

In [62]:
dfs_frutas = GetSerieProducto(df_frutas, group)

In [63]:
dfs_frutas.sample(3)

,Momento,Sector,Producto,Factor
75,2015-11-01,Frutillar,Frutas,1.053532
227,2018-05-01,Osorno,Frutas,0.888146
231,2018-06-01,Llanquihue,Frutas,0.926387


#### 5.8 Lacteos

In [64]:
df_lacteos = df[df.Producto=="Lacteos"]

In [65]:
group = df_lacteos.groupby(['Momento', 'Sector', 'Producto'])

In [66]:
dfs_lacteos = GetSerieProducto(df_lacteos, group)

In [67]:
dfs_lacteos.sample(3)

,Momento,Sector,Producto,Factor
44,2015-04-01,Puerto Varas,Lacteos,1.014709
19,2014-11-01,Puerto Varas,Lacteos,0.989649
5,2014-09-01,Frutillar,Lacteos,0.963179


#### 5.9 Pan

In [68]:
df_pan = df[df.Producto=="Pan"]

In [69]:
group = df_pan.groupby(['Momento', 'Sector', 'Producto'])

In [70]:
dfs_pan = GetSerieProducto(df_pan, group)

In [71]:
dfs_pan.sample(3)

,Momento,Sector,Producto,Factor
162,2017-04-01,Osorno,Pan,0.977643
67,2015-09-01,Osorno,Pan,0.929605
190,2017-10-01,Frutillar,Pan,0.975130


#### 5.10 Huevos

In [72]:
df_huevos = df[df.Producto=="Huevos"]

In [73]:
group = df_huevos.groupby(['Momento', 'Sector', 'Producto'])

In [74]:
dfs_huevos = GetSerieProducto(df_huevos, group)

In [75]:
dfs_huevos.sample(3)

,Momento,Sector,Producto,Factor
156,2017-03-01,Llanquihue,Huevos,0.975529
141,2016-12-01,Llanquihue,Huevos,1.009943
137,2016-11-01,Osorno,Huevos,0.993475


#### 5.11 Legumbre

In [76]:
df_legumbre = df[df.Producto=="Legumbre"]

In [77]:
group = df_legumbre.groupby(['Momento', 'Sector', 'Producto'])

In [78]:
dfs_legumbre = GetSerieProducto(df_legumbre, group)

In [79]:
dfs_legumbre.sample(3)

,Momento,Sector,Producto,Factor
148,2017-01-01,Puerto Montt,Legumbre,0.999211
194,2017-10-01,Puerto Varas,Legumbre,1.027233
201,2017-12-01,Llanquihue,Legumbre,0.967537


### 6. Creamos dataset de precios

In [80]:
pd.concat([dfs_legumbre, 
           dfs_huevos, 
           dfs_pan, 
           dfs_lacteos, 
           dfs_frutas, 
           dfs_hortaliza, 
           dfs_carnes, 
           dfs_azucar, 
           dfs_cereales, 
           dfs_harina, 
           dfs_aceite]).to_csv("output/precios_productos.csv", index=False)